In [1]:
#
# Working through
# https://github.com/salvadorgarciamunoz/kipet/blob/master/kipet/examples/Ex_7_conc_input_conf_k_aug.py
#

In [2]:
from __future__ import print_function
from kipet.library.TemplateBuilder import *
from kipet.library.PyomoSimulator import *
from kipet.library.ParameterEstimator import *
from kipet.library.VarianceEstimator import *
from kipet.library.data_tools import *
import matplotlib.pyplot as plt
import os
import sys
import inspect
import six
import pandas as pd

In [3]:
filename = '../data_sets/Ex_1_C_data.txt'
D_frame = read_file(filename, directory = '../data_sets')

read dir : /home/paperspace/learn_kipet/kipet_examples/../data_sets/../data_sets/Ex_1_C_data.txt


In [4]:
D_frame.shape

(300, 3)

In [5]:
builder = TemplateBuilder()    
components = {'A':1e-3,'B':0,'C':0}
builder.add_mixture_component(components)
builder.add_parameter('k1',bounds=(0.0,5.0))
builder.add_parameter('k2',bounds=(0.00,2.0))
builder.add_concentration_data(D_frame)

               A         B         C
0.0000  0.001027  0.000000  0.000013
0.0333  0.001015  0.000002  0.000021
0.0667  0.001007  0.000013  0.000018
0.1000  0.000993  0.000025  0.000014
0.1334  0.000982  0.000035  0.000012
...          ...       ...       ...
9.8353  0.000103  0.000043  0.000859
9.8686  0.000104  0.000042  0.000857
9.9020  0.000103  0.000041  0.000859
9.9353  0.000101  0.000043  0.000860
9.9687  0.000103  0.000038  0.000863

[300 rows x 3 columns]


In [14]:
# define explicit system of ODEs
def rule_odes(m,t):
    exprs = dict()
    exprs['A'] = -m.P['k1']*m.Z[t,'A']
    exprs['B'] = m.P['k1']*m.Z[t,'A']-m.P['k2']*m.Z[t,'B']
    exprs['C'] = m.P['k2']*m.Z[t,'B']
    return exprs

builder.set_odes_rule(rule_odes)
opt_model = builder.create_pyomo_model(0.0,10.0)

In [15]:
sigmas = {'A':1e-10,'B':1e-11,'C':1e-8}
p_estimator = ParameterEstimator(opt_model)
p_estimator.apply_discretization('dae.collocation',nfe=100,ncp=3,scheme='LAGRANGE-RADAU')

    number of finite elements specified in apply. The larger number of finite
    elements will be used.


In [16]:
options = dict()
#options['nlp_scaling_method'] = 'gradient-based'
#options['bound_relax_factor'] = 0
# finally we run the optimization
results_pyomo = p_estimator.run_opt('k_aug',
                                    variances=sigmas,
                                  tee=True,
                                  solver_opts = options,
                                  covariance=True)

        0.62 seconds required to write file
        0.62 seconds required for presolve
Ipopt 3.12: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:    23403
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     2700

Total number of variables............................:     5405
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        2
                     variables with only upper bounds:        0
T

W[K_AUG]...	[K_AUG_ASL]No n_rhs declared
W[K_AUG]...	[K_AUG_ASL]Target log10mu:= -11.
W[K_AUG]...	[K_AUG_ASL]No f_timestamp suffix declared, Fallback to default writing mode.
I[K_AUG]...	[K_AUG_ASL] Filename for dot_sens dot_in_.in


[K_AUG] written by D.T. @2018

: I[K_AUG]...	[K_AUG_ASL]File read succesfull
I[K_AUG]...	[K_AUG_ASL]Number of Right hand sides 0
I[K_AUG]...	[K_AUG_ASL]Number of Right hand sides: 0
I[K_AUG]...	[K_AUG_ASL]Number of variables       : 5405
I[K_AUG]...	[K_AUG_ASL]Number of constraints     : 5403
I[K_AUG]...	[K_AUG_ASL]Number of valid n_dof     : 2
I[K_AUG]...	[ADJUST_MU]log10(mu) computed=-4.894514883980449 at var_i=1800
I[K_AUG]...	[ADJUST_MU]log10(mu) close to the target	-4.8977593836075135 at var_i=1801
I[K_AUG]...	[FIND_INEQUALITIES]summary: eq: 5403, leq: 0, geq: 0 
I[K_AUG]...	[K_AUG_ASL]Nonzeroes in the sparse Jacobian 23403
I[K_AUG]...	[GET_HESS_ASL]Objective found
I[K_AUG]...	[GET_HESS_ASL]Nonzeroes in the sparse hessian 2700
I[K_AUG]...	[GET_HESS_ASL]Minimization problem detected
I[K_AUG]...	[GET_HESS_ASL]Current objective 1172.675059
I[K_AUG]...	[GET_HESS_ASL]Missing nz in the Hessian of the Lag: 4505
I[K_AUG]...	[K_AUG_ASL]Barrier term added.
I[K_AUG]...	[K_AUG_ASL]MC19 scalin

W[K_AUG]...	[MUMPS_DRIVER]icntl 14 > 200
W[K_AUG]...	[INERTIA_STRATEGY]Wrong inertia(neig < m).
W[K_AUG]...	[INERTIA_STRATEGY]Attempting to make i_pert->d_c > 0.


I[K_AUG]...	[MUMPS_DRIVER]Reallocating Memory (50)

 On return from DMUMPS, INFOG(1)=              -9
 On return from DMUMPS, INFOG(2)=          180325
I[K_AUG]...	[MUMPS_DRIVER]Reallocating Memory (100)

 On return from DMUMPS, INFOG(1)=              -9
 On return from DMUMPS, INFOG(2)=          101725
I[K_AUG]...	[MUMPS_DRIVER]Reallocating Memory (200)

I[K_AUG]...	[MUMPS_DRIVER]n_neig = 5403
I[K_AUG]...	[INERTIA_STRATEGY]Inertia check successful.
I[K_AUG]...	[MUMPS_DRIVER]: Ratio of norm of scaled residuals (reported); 1.982329e-15 
I[K_AUG]...	[MUMPS_DRIVER]Accuracy at an acceptable level.

W[K_AUG]...	[MUMPS_DRIVER]Inertia check OK neig=5403, (neig == m).
I[K_AUG]...	[K_AUG_ASL]Linear solver done. 
I[K_AUG]...	[K_AUG_ASL]Timings..Ev&Assem 0.198188, Fact 0.811545, Overall 0.82302.
Done solving building reduce hessian
4 hessian size
Parameter variances:  [1.47427311e-07 4.95992665e-06]

Confidence intervals:
k1 (0.22630611417543584,0.22707403946042443)
k2 (1.0280192738950906,1.0324

In [17]:
print("The estimated parameters are:")
for k,v in six.iteritems(results_pyomo.P):
    print(k, v)

The estimated parameters are:
k1 0.22669007681793013
k2 1.030246363173192
